In [1]:
import os
import tempfile

import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from tensorflow import keras

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [3]:
%load_ext tensorboard

In [4]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape = (28, 28)),
  keras.layers.Reshape(target_shape = (28, 28, 1)),
  keras.layers.Conv2D(filters = 12, kernel_size = (3, 3), activation = tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size = (2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs = 4,
  validation_split = 0.1,
)

Epoch 1/4
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2891 - accuracy: 0.9194 - val_loss: 0.1114 - val_accuracy: 0.9717
Epoch 2/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1061 - accuracy: 0.9698 - val_loss: 0.0841 - val_accuracy: 0.9788
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0792 - accuracy: 0.9767 - val_loss: 0.0692 - val_accuracy: 0.9815
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0665 - accuracy: 0.9809 - val_loss: 0.0636 - val_accuracy: 0.9815


In [5]:
# Evaluate baseline test accuracy and save the model for later usage.

_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose = 0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer = False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9786999821662903
Saved baseline model to: /tmp/tmpsye36u14.h5


In [6]:
# start the model with 50% sparsity (50% zeros in weights) and end with 80% sparsity.

In [7]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity = 0.50,
                                                               final_sparsity = 0.80,
                                                               begin_step = 0,
                                                               end_step = end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

model_for_pruning.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


#### Train and evaluate the model against baseline

Fine tune with pruning for two epochs.

tfmot.sparsity.keras.UpdatePruningStep is required during training, and tfmot.sparsity.keras.PruningSummaries provides logs for tracking progress and debugging.

In [8]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir = logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size = batch_size, epochs = epochs, validation_split = validation_split,
                  callbacks = callbacks)

Epoch 1/2
422/422 [==============================] - 6s 13ms/step - loss: 0.0815 - accuracy: 0.9770 - val_loss: 0.0852 - val_accuracy: 0.9783
Epoch 2/2
422/422 [==============================] - 3s 8ms/step - loss: 0.0874 - accuracy: 0.9756 - val_loss: 0.0742 - val_accuracy: 0.9817


In [9]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose = 0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9786999821662903
Pruned test accuracy: 0.9750999808311462


In [10]:
%tensorboard --logdir={logdir}